# Calculating SOAP descriptors for zeolites

In this notebook, we will calculate the SOAP descriptors for IZA zeolites using the package `dscribe`.

In [1]:
import os
import warnings
import pandas as pd
import numpy as np
import amd
from ase.io import read
from dscribe.descriptors import SOAP

warnings.filterwarnings("ignore")

## Calculation of the SOAP descriptors

In [2]:
species = ["Si", "O"]
r_cut = 6.0
n_max = 8
l_max = 6

soap = SOAP(
    species=species,
    periodic=True,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    average="inner",
)

In [3]:
path = os.path.abspath('../dsets/iza_experimental')
files = sorted([f for f in os.listdir(path) if f.endswith(".cif")])

## Timing of SOAP vectors + I/O by ASE

Considering the time to read the CIF files using ASE's `read` function in the timing:

In [4]:
%%timeit

soap_vecs = []
for f in files:
    atoms = read(os.path.join(path, f))
    soap_vecs.append(soap.create(atoms))
    
soap_vecs = np.array(soap_vecs)

dotprod = (soap_vecs @ soap_vecs.T)
norms = np.diag(dotprod)
kernel = dotprod / np.sqrt(norms.reshape(-1, 1) * norms.reshape(1, -1))
dm = np.sqrt(2 - 2 * kernel)

19.3 s ± 175 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Timing of creating SOAP vectors without the I/O from ASE

If we separate the time to read the SOAP vectors vs. the time to perform the distance calculations

In [5]:
dset = [
    read(os.path.join(path, f))
    for f in files
]

In [6]:
%%timeit

soap_vecs = []
for atoms in dset:
    soap_vecs.append(soap.create(atoms))
    
soap_vecs = np.array(soap_vecs)

dotprod = (soap_vecs @ soap_vecs.T)
norms = np.diag(dotprod)
kernel = dotprod / np.sqrt(norms.reshape(-1, 1) * norms.reshape(1, -1))
dm = np.sqrt(2 - 2 * kernel)

5.9 s ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Calculation of AMD

The distance matrix using AMD is computed including the time to read the files.
(note: because the code is compiled using numba, the first execution is slower than usual)

In [8]:
%%timeit

k = 100
path = os.path.abspath('../dsets/iza_experimental')
_dm = amd.compare(path, by='AMD', k=k)

1.82 s ± 24.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
